<a href="https://colab.research.google.com/github/thieuhy/AgenticAI_Business_SJSU/blob/Agentic-AI-in-Marketing/Group_8_Agentic_AI_in_Marketing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README

**How to Run:**  
1. Open and run all notebook cells from top to bottom.  
2. The agent will:
   - Read campaign data from `marketing_campaign_dataset.xlsx`
   - Compute per-channel performance (CTR, CVR, CPA)
   - Allocate daily budgets using a heuristic + guardrails
   - Save outputs to:
     - `agent_decisions_log.csv` — daily decisions and rationales  
     - `agent_allocations.csv` — budget allocations  
     - `summary_comparison.csv` — baseline vs. optimized results  

**Assumptions:**  
- 3–4 channels (e.g., Search, Social, Display)  
- ~14–30 days of data from the Excel file  
- Optimization aims to improve conversions with fairness (±20% cap, 10% floor)  
- No use of personal or sensitive data  

**Results Snapshot (Example):**

| Metric | Baseline | Optimized | Δ (%) |
|--------|-----------|------------|-------|
| Total Conversions | 118 | 131 | +11.0% |
| Avg CPA | 13.2 | 12.1 | -8.3% |
| CTR | 2.4% | 2.6% | +8.5% |


**Observation:**  
The heuristic agent increased conversions while maintaining exploration and respecting daily guardrails.

In [ ]:
# --- install dependencies (run once per session)
!pip -q install langchain langchain_openai openai

# --- imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os
from getpass import getpass

# --- ask each user to enter their own key securely
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

# --- setup the model + embeddings
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

print("✅ LangChain + OpenAI setup complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 1.6 MB/s eta 0:00:00
Enter your OpenAI API key: ··········
✅ LangChain + OpenAI setup complete.


In [ ]:
"""
Ad Optimization Agent (3 channels): equal-start + explore/exploit + guardrails
- Baseline: equal split daily
- Agent: shift +10–20% toward prior-day top performer by CVR (CTR fallback)
- Guardrails: per-day change cap (±20%), min floor for all channels, never allocate 0% > 2 days
- Evaluation: total conversions, total clicks, CPA vs baseline
"""

"""
Guardrails:
- Limit budget changes to ±20% per day (prevents big swings)
- Maintain a minimum 15% spend per channel (keeps learning active)
- No channel can stay near 0% for more than 2 days (avoids abandonment)
- Always re-normalize allocations to sum to 100%
- Use CVR to choose top performer (CTR fallback if clicks are low)
"""

import os
from dataclasses import dataclass
import math
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
np.random.seed(7)

# =========================
# CONFIG
# =========================
CHANNELS = ["Search", "Social", "Display"]
DAYS = 21
DAILY_BUDGET = 3000.0

# Heuristic knobs
SHIFT_PCT = 0.15             # shift 10–20% toward top (set 0.10–0.20)
MIN_FLOOR_PCT = 0.15         # never allocate below 15% (keeps learning alive)
MAX_DELTA_PCT = 0.20         # per-day change cap (+/-20%)
ZERO_MAX_STREAK = 2          # never 0% for > 2 consecutive days

# Eval metric priority (CVR with CTR fallback)
PRIORITY_METRIC = "CVR"      # computed as conversions / clicks (fallback CTR if clicks==0)

OUTDIR = "./ad_agent_outputs"
os.makedirs(OUTDIR, exist_ok=True)

# =========================
# UTILITIES
# =========================
@dataclass
class DayOutcome:
    date: str
    channel: str
    spend: float
    impressions: float
    clicks: float
    conversions: float

def summarize(df: pd.DataFrame) -> pd.DataFrame:
    g = df.groupby("channel").agg(
        spend=("spend", "sum"),
        impressions=("impressions", "sum"),
        clicks=("clicks", "sum"),
        conversions=("conversions", "sum"),
    )
    g.loc["TOTAL"] = g.sum(numeric_only=True)
    g["CTR"] = g["clicks"] / g["impressions"]
    g["CVR"] = g["conversions"] / g["clicks"]
    g["CPA"] = g["spend"] / g["conversions"]
    return g

def safe_div(num, den):
    return num / den if den > 0 else 0.0

# =========================
# DATA INPUT
# =========================
"""
Option A (default): create a simple synthetic dataset with latent rates.
Option B: read your own CSV/Excel with columns: date,channel,impressions,clicks,conversions.
   - To use Option B, set USE_EXTERNAL=True and provide a path (CSV or XLSX).
"""
USE_EXTERNAL = False
EXTERNAL_PATH = "./marketing_campaign_dataset.xlsx"  # or .csv

def build_dates(n_days: int):
    start = datetime.today().date() - timedelta(days=n_days)
    return [start + timedelta(days=i) for i in range(n_days)]

def make_synthetic_latent():
    """Latent CTR/CVR/CPM per channel, slightly noisy by day."""
    profiles = {
        "Search":  {"ctr": 0.045, "cvr": 0.045, "cpm": 8.0},
        "Social":  {"ctr": 0.012, "cvr": 0.025, "cpm": 5.0},
        "Display": {"ctr": 0.006, "cvr": 0.012, "cpm": 3.5},
    }
    dates = build_dates(DAYS)
    rows = []
    for d in dates:
        for ch in CHANNELS:
            ctr = max(0.0001, np.random.normal(profiles[ch]["ctr"], profiles[ch]["ctr"]*0.12))
            cvr = max(0.0001, np.random.normal(profiles[ch]["cvr"], profiles[ch]["cvr"]*0.12))
            cpm = max(1.0, np.random.normal(profiles[ch]["cpm"], profiles[ch]["cpm"]*0.08))
            rows.append({"date": d.isoformat(), "channel": ch, "ctr": ctr, "cvr": cvr, "cpm": cpm})
    return pd.DataFrame(rows)

def simulate_day(latent_df: pd.DataFrame, date_str: str, alloc_pct: dict) -> pd.DataFrame:
    """Use CPM to convert budget to impressions, then CTR->clicks, CVR->conversions."""
    day_params = latent_df[latent_df["date"] == date_str]
    out = []
    for ch in CHANNELS:
        spend = DAILY_BUDGET * alloc_pct[ch]
        cpm = float(day_params.loc[day_params["channel"] == ch, "cpm"].iloc[0])
        ctr = float(day_params.loc[day_params["channel"] == ch, "ctr"].iloc[0])
        cvr = float(day_params.loc[day_params["channel"] == ch, "cvr"].iloc[0])
        imps = (spend / cpm) * 1000.0
        clicks = imps * ctr
        conv = clicks * cvr
        out.append(DayOutcome(date_str, ch, spend, imps, clicks, conv).__dict__)
    return pd.DataFrame(out)

def load_external_perf(path: str) -> pd.DataFrame:
    """
    Expect columns: date,channel,impressions,clicks,conversions
    We will estimate CPM per channel-day to translate budget to impressions realistically.
    """
    if path.endswith(".xlsx"):
        raw = pd.read_excel(path)
    else:
        raw = pd.read_csv(path)
    # clean and conform
    need = {"date","channel","impressions","clicks","conversions"}
    assert need.issubset(set(raw.columns)), f"Missing required columns: {need - set(raw.columns)}"
    df = raw.copy()
    # Keep last DAYS * 3 rows if too large (or aggregate by date/channel)
    df["date"] = pd.to_datetime(df["date"]).dt.date.astype(str)
    # Ensure exactly DAYS per channel; if not, sample or pad
    # For simplicity, just take most recent DAYS per channel if more available
    perf = []
    for ch in CHANNELS:
        sub = df[df["channel"].str.lower() == ch.lower()].copy()
        if sub.empty:
            raise ValueError(f"No rows found for channel {ch} in external file.")
        sub = sub.sort_values("date").tail(DAYS)
        # Estimate CTR/CVR to use as day parameters; derive CPM by an assumed $5 baseline scaled by density
        sub["ctr"] = sub["clicks"] / sub["impressions"].replace(0, np.nan)
        sub["ctr"] = sub["ctr"].fillna(0.001).clip(lower=0.0001)
        sub["cvr"] = sub["conversions"] / sub["clicks"].replace(0, np.nan)
        sub["cvr"] = sub["cvr"].fillna(0.01).clip(lower=0.0001)
        # crude CPM proxy: lower impressions per unit click ⇒ higher CPM; bound to [3,12]
        density = (sub["impressions"] / (sub["clicks"].replace(0,np.nan))).fillna(1000)
        sub["cpm"] = (density / density.median()) * 6
        sub["cpm"] = sub["cpm"].clip(3, 12)
        sub["channel"] = ch
        perf.append(sub[["date","channel","ctr","cvr","cpm"]])
    return pd.concat(perf, ignore_index=True)

if USE_EXTERNAL:
    latent = load_external_perf(EXTERNAL_PATH)
else:
    latent = make_synthetic_latent()

latent.to_csv(os.path.join(OUTDIR, "latent_params.csv"), index=False)

DATES = sorted(latent["date"].unique().tolist())
assert len(DATES) == DAYS, "Unexpected number of days in latent frame."

# =========================
# BASELINE: equal split every day
# =========================
equal_alloc = {ch: 1.0/len(CHANNELS) for ch in CHANNELS}
baseline_runs = []
for d in DATES:
    baseline_runs.append(simulate_day(latent, d, equal_alloc))
baseline_df = pd.concat(baseline_runs, ignore_index=True)
baseline_df.to_csv(os.path.join(OUTDIR, "baseline_equal.csv"), index=False)

# =========================
# AGENT: explore/exploit with guardrails
# =========================
def compute_scores(df_prev_day: pd.DataFrame) -> dict:
    g = df_prev_day.groupby("channel").sum(numeric_only=True)
    scores = {}
    for ch in CHANNELS:
        clicks = float(g.loc[ch, "clicks"]) if ch in g.index else 0.0
        convs = float(g.loc[ch, "conversions"]) if ch in g.index else 0.0
        imps  = float(g.loc[ch, "impressions"]) if ch in g.index else 0.0
        if clicks > 0:
            cvr = convs / clicks
            scores[ch] = cvr
        else:
            ctr = clicks / imps if imps > 0 else 0.0
            scores[ch] = ctr
    return scores  # higher is better

def apply_guardrails(prev_alloc: dict, proposed: dict, zero_streaks: dict) -> dict:
    guarded = {}
    # 1) per-day delta cap and min floor
    for ch in CHANNELS:
        prev = prev_alloc[ch]
        prop = proposed[ch]
        delta = np.clip(prop - prev, -MAX_DELTA_PCT, MAX_DELTA_PCT)
        prop = prev + delta
        prop = max(prop, MIN_FLOOR_PCT)   # exploration floor
        guarded[ch] = prop
    # 2) never 0% for >2 days — bump channel if streak exceeded
    for ch in CHANNELS:
        if zero_streaks[ch] > ZERO_MAX_STREAK and guarded[ch] <= 0.001:
            guarded[ch] = max(guarded[ch], MIN_FLOOR_PCT)
    # 3) renormalize to 1.0
    s = sum(guarded.values())
    for ch in CHANNELS:
        guarded[ch] = guarded[ch] / s
    return guarded

# State
agent_alloc = equal_alloc.copy()
agent_logs = []
alloc_history = []
decision_log = []  # rationale strings
zero_streaks = {ch: 0 for ch in CHANNELS}

# Day 0
d0 = DATES[0]
day0 = simulate_day(latent, d0, agent_alloc)
agent_logs.append(day0)
alloc_history.append({"date": d0, **agent_alloc})
decision_log.append({
    "date": d0,
    "decision": "Init equal split",
    "reason": "Start exploration with equal allocation across channels."
})

# Subsequent days
for i in range(1, len(DATES)):
    prev_day = agent_logs[-1]
    scores = compute_scores(prev_day)
    top = max(scores, key=scores.get)

    # Propose: shift SHIFT_PCT toward top, take evenly from others
    proposed = agent_alloc.copy()
    add = SHIFT_PCT
    others = [c for c in CHANNELS if c != top]
    for o in others:
        proposed[o] -= add / len(others)
    proposed[top] += add

    # Guardrails
    guarded = apply_guardrails(agent_alloc, proposed, zero_streaks)

    # Log zero streaks (if any channel effectively zero)
    for ch in CHANNELS:
        if guarded[ch] <= 0.001:
            zero_streaks[ch] += 1
        else:
            zero_streaks[ch] = 0

    # Human-readable rationale
    metric_used = "CVR-pref (CTR fallback)"
    reason = (
        f"{top} up by ~{int(add*100)}% due to higher {metric_used} on {DATES[i-1]}. "
        f"Applied guardrails: ±{int(MAX_DELTA_PCT*100)}% cap, "
        f"min floor {int(MIN_FLOOR_PCT*100)}%, zero-streak≤{ZERO_MAX_STREAK}."
    )

    # Apply and simulate
    agent_alloc = guarded
    di = DATES[i]
    out = simulate_day(latent, di, agent_alloc)
    agent_logs.append(out)
    alloc_history.append({"date": di, **agent_alloc})
    decision_log.append({"date": di, "decision": f"Boost {top}", "reason": reason})

agent_df = pd.concat(agent_logs, ignore_index=True)
alloc_hist_df = pd.DataFrame(alloc_history)
decisions_df = pd.DataFrame(decision_log)

# =========================
# EVALUATION
# =========================
sum_base = summarize(baseline_df)
sum_agent = summarize(agent_df)

comparison = pd.concat(
    {"Baseline_Equal": sum_base.loc[["Search","Social","Display","TOTAL"]],
     "Agent": sum_agent.loc[["Search","Social","Display","TOTAL"]]},
    axis=1
)

# Save artifacts
baseline_df.to_csv(os.path.join(OUTDIR, "baseline_outcomes.csv"), index=False)
agent_df.to_csv(os.path.join(OUTDIR, "agent_outcomes.csv"), index=False)
alloc_hist_df.to_csv(os.path.join(OUTDIR, "agent_allocations.csv"), index=False)
decisions_df.to_csv(os.path.join(OUTDIR, "agent_decisions_log.csv"), index=False)
comparison.to_csv(os.path.join(OUTDIR, "summary_comparison.csv"))

# Print snapshots
print("=== Summary (Totals) ===")
print(comparison.round(3))
print("\n=== Sample Decisions ===")
print(decisions_df.head(8))
print(f"\nArtifacts saved to: {OUTDIR}")


=== Summary (Totals) ===
        Baseline_Equal                                                      \
                 spend   impressions      clicks conversions    CTR    CVR   
channel                                                                      
Search         21000.0  2.565404e+06  116151.630    5190.898  0.045  0.045   
Social         21000.0  4.179256e+06   49181.759    1201.434  0.012  0.024   
Display        21000.0  6.070259e+06   36219.425     413.697  0.006  0.011   
TOTAL          63000.0  1.281492e+07  201552.814    6806.029  0.016  0.034   

                     Agent                                                     \
            CPA      spend  impressions      clicks conversions    CTR    CVR   
channel                                                                         
Search    4.046  44747.465  5463437.021  245784.885   11029.516  0.045  0.045   
Social   17.479   9126.267  1815350.565   21549.472     523.602  0.012  0.024   
Display  50.762   9126.

In [ ]:
pip install langgraph langchain langchain_openai pydantic

In [ ]:
import os
import operator
from typing import TypedDict, Annotated, List

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END

from google.colab import userdata

# --- ask each user to enter their own key securely
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

# 1. Define the Agent State
# This represents the information passed between nodes in the graph
class AgentState(TypedDict):
    """
    Represents the state of our ad optimization agent.
    - messages: A list of messages/history.
    - campaign_data: The current (simulated) ad campaign metrics.
    - next_action: The recommended action from the analysis.
    """
    messages: Annotated[List[BaseMessage], operator.add]
    campaign_data: dict
    next_action: str

# 2. Initialize LLM
# We'll use a powerful model for the reasoning engine
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ LangChain + OpenAI setup complete.")

In [ ]:
# --- Simulated Tools for Ad Optimization ---

@tool
def adjust_bid(campaign_id: str, new_bid_amount: float) -> str:
    """Adjusts the bid for a specific ad campaign to the new amount."""
    if new_bid_amount > 5.00:
        return f"Bid for Campaign '{campaign_id}' adjusted to ${new_bid_amount:.2f}. (Simulated: High bid warning!)"
    return f"Bid for Campaign '{campaign_id}' adjusted to ${new_bid_amount:.2f} successfully."

@tool
def pause_ad_group(ad_group_id: str) -> str:
    """Pauses an underperforming ad group by its ID."""
    return f"Ad Group '{ad_group_id}' paused successfully due to poor performance."

@tool
def request_new_creatives(campaign_id: str) -> str:
    """Requests new ad creative assets from the creative team."""
    return f"New creative request submitted for Campaign '{campaign_id}'. Awaiting design feedback."

ad_optimization_tools = [adjust_bid, pause_ad_group, request_new_creatives]

In [ ]:
def fetch_campaign_data(state: AgentState) -> dict:
    """Simulates fetching the latest campaign data."""
    print("--- FETCHING DATA ---")
    # In a real scenario, this would call an Ad API (e.g., Google Ads, Meta)
    # This is a fixed, simulated dataset for the workshop
    simulated_data = {
        "Campaign-2024-Q4": {
            "Budget": 1000, "Spend": 850, "Impressions": 50000,
            "Clicks": 500, "Conversions": 5, "CPA": 170.00, "Target_CPA": 50.00,
            "Ad_Groups": {
                "AG-101": {"Status": "Active", "CPA": 25.00, "Bid": 2.50},
                "AG-102": {"Status": "Active", "CPA": 450.00, "Bid": 3.00} # Poor performer
            }
        }
    }

    analysis_prompt = (
        "Ad Campaign Data for Optimization:\n"
        f"{simulated_data}\n\n"
        "**Optimization Goal:** Reduce overall CPA (Cost Per Acquisition) to be closer to the Target CPA ($50.00) "
        "and maximize conversions. Analyze the data and recommend the *single best action* "
        "using one of the provided tools (adjust_bid, pause_ad_group, request_new_creatives). "
        "Your final output should be ONLY the recommended action as a message for the user, "
        "or a clear instruction for the next internal step."
    )

    # Prepend the analysis prompt to the messages for the LLM
    new_messages = [HumanMessage(content=analysis_prompt)]

    return {"messages": new_messages, "campaign_data": simulated_data}

def agent_reasoning(state: AgentState) -> dict:
    """The LLM reasons and decides the next step (tool call or final answer)."""
    print("--- AGENT REASONING ---")

    # Bind the tools to the LLM
    llm_with_tools = llm.bind_tools(ad_optimization_tools)

    # Define a system prompt to guide the LLM's role
    system_prompt = (
        "You are an expert Ad Campaign Optimization Agent. "
        "Your goal is to analyze the provided campaign data and decide the optimal next action. "
        "You must use a tool if an optimization is possible. "
        "If no tool is needed or you have executed a tool, provide a final, concise update."
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("placeholder", "{messages}")
    ])

    # Run the LLM to get a decision
    chain = prompt | llm_with_tools
    response = chain.invoke(state)

    # Determine if a tool call was made
    if response.tool_calls:
        # If a tool is called, the next step is to execute it.
        return {"messages": [response], "next_action": "call_tool"}
    else:
        # If no tool is called, the reasoning is the final response.
        return {"messages": [response], "next_action": "FINISH"}


def execute_tools(state: AgentState) -> dict:
    """Executes the tool call(s) decided by the agent_reasoning step."""
    print("--- EXECUTING TOOL ---")

    tool_calls = state["messages"][-1].tool_calls
    tool_results = []

    # Find and execute the function corresponding to the tool call
    for call in tool_calls:
        tool_name = call["name"]
        tool_args = call["args"]
        tool_call_id = call["id"] # Get the tool call ID

        # Simple lookup and execution (in a real app, use the ToolExecutor)
        tool_to_run = next(
            (t for t in ad_optimization_tools if t.name == tool_name), None
        )

        if tool_to_run:
            try:
                result = tool_to_run.invoke(tool_args)
                # Format the tool result as a ToolMessage
                tool_results.append({
                    "type": "tool_output",
                    "tool_call_id": tool_call_id, # Include the tool call ID
                    "content": result
                })
            except Exception as e:
                 # Handle tool execution errors and return an error message
                 tool_results.append({
                    "type": "tool_output",
                    "tool_call_id": tool_call_id, # Include the tool call ID
                    "content": f"Error executing tool {tool_name}: {e}"
                })


    # Add tool results to the state for the LLM to process next
    # Need to format the tool results into a list of BaseMessage
    tool_messages = [HumanMessage(content=str(r), name="tool_execution") for r in tool_results]

    return {"messages": tool_messages}

def decide_next_step(state: AgentState) -> str:
    """Conditional edge: decides whether to continue analysis or finish."""
    print(f"--- DECIDING NEXT STEP ---")
    print(f"state['next_action']: {state.get('next_action')}") # Use .get() for safety

    next_action = state.get("next_action")

    if next_action == "call_tool":
        print("Returning 'call_tool'") # Corrected return value
        return "call_tool"
    elif next_action == "FINISH":
        print("Returning 'end'")
        return "end"
    else:
        # After tool execution, go back to reasoning to summarize the result
        print("Returning 'agent_reasoning'")
        return "agent_reasoning"

In [ ]:
from langchain_core.messages import HumanMessage
import pandas as pd, re

def fetch_campaign_data(state: AgentState) -> dict:
    print("--- FETCHING DATA ---")
    df = pd.read_excel("/content/marketing_campaign_dataset.xlsx")
    df.columns = [re.sub(r'[^a-z0-9]+', '_', c.strip().lower()) for c in df.columns]
    df = df.head(50)

    # map to your actual headers
    campaign_col = "campaign_id"
    adgroup_col  = "campaign_type"
    clicks_col   = "clicks"          # used as a conversions proxy
    impr_col     = "impressions"

    simulated_data = {}
    for camp, g in df.groupby(campaign_col):
        entry = {
            "Impressions": int(g[impr_col].sum()) if impr_col in g else None,
            "Clicks": int(g[clicks_col].sum()) if clicks_col in g else None,
            "Conversions": int(g[clicks_col].sum()) if clicks_col in g else None,
            "Target_CPA": 50.00,
            "Ad_Groups": {
                str(ag): {
                    "Status": "Active",
                    "CPA": None,
                    "Bid": None
                }
                for ag, _ in g.groupby(adgroup_col)
            }
        }
        simulated_data[str(camp)] = entry

    analysis_prompt = (
        "Ad Campaign Data for Optimization:\n"
        f"{simulated_data}\n\n"
        "**Optimization Goal:** Reduce overall CPA to be closer to Target CPA ($50.00) "
        "and maximize conversions. Recommend the *single best action* "
        "using adjust_bid, pause_ad_group, or request_new_creatives."
    )
    new_messages = [HumanMessage(content=analysis_prompt)]
    return {"messages": new_messages, "campaign_data": simulated_data}


def agent_reasoning(state: AgentState) -> dict:
    """The LLM reasons and decides the next step (tool call or final answer)."""
    print("--- AGENT REASONING ---")

    # Bind the tools to the LLM
    llm_with_tools = llm.bind_tools(ad_optimization_tools)

    # Define a system prompt to guide the LLM's role
    system_prompt = (
        "You are an expert Ad Campaign Optimization Agent. "
        "Your goal is to analyze the provided campaign data and decide the optimal next action. "
        "You must use a tool if an optimization is possible. "
        "If no tool is needed or you have executed a tool, provide a final, concise update."
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("placeholder", "{messages}")
    ])

    # Run the LLM to get a decision
    chain = prompt | llm_with_tools
    response = chain.invoke(state)

    # Determine if a tool call was made
    if response.tool_calls:
        # If a tool is called, the next step is to execute it.
        return {"messages": [response], "next_action": "call_tool"}
    else:
        # If no tool is called, the reasoning is the final response.
        return {"messages": [response], "next_action": "FINISH"}


def execute_tools(state: AgentState) -> dict:
    """Executes the tool call(s) decided by the agent_reasoning step."""
    print("--- EXECUTING TOOL ---")

    tool_calls = state["messages"][-1].tool_calls
    tool_results = []

    # Find and execute the function corresponding to the tool call
    for call in tool_calls:
        tool_name = call["name"]
        tool_args = call["args"]
        tool_call_id = call["id"] # Get the tool call ID

        # Simple lookup and execution (in a real app, use the ToolExecutor)
        tool_to_run = next(
            (t for t in ad_optimization_tools if t.name == tool_name), None
        )

        if tool_to_run:
            try:
                result = tool_to_run.invoke(tool_args)
                # Format the tool result as a ToolMessage
                tool_results.append(ToolMessage(
                    content=result,
                    tool_call_id=tool_call_id # Include the tool call ID
                ))
            except Exception as e:
                 # Handle tool execution errors and return an error message
                 tool_results.append(ToolMessage(
                    content=f"Error executing tool {tool_name}: {e}",
                    tool_call_id=tool_call_id # Include the tool call ID
                ))


    # Add tool results to the state for the LLM to process next
    return {"messages": tool_results}

def decide_next_step(state: AgentState) -> str:
    """Conditional edge: decides whether to continue analysis or finish."""
    print(f"--- DECIDING NEXT STEP ---")
    print(f"state['next_action']: {state.get('next_action')}") # Use .get() for safety

    next_action = state.get("next_action")

    if next_action == "call_tool":
        print("Returning 'call_tool'") # Corrected return value
        return "call_tool"
    elif next_action == "FINISH":
        print("Returning 'end'")
        return "end"
    else:
        # After tool execution, go back to reasoning to summarize the result
        print("Returning 'agent_reasoning'")
        return "agent_reasoning"

In [ ]:
# 5. Build the LangGraph Workflow
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("fetch_data", fetch_campaign_data)
workflow.add_node("agent_reasoning", agent_reasoning)
workflow.add_node("execute_tools", execute_tools)

# Set the start point
workflow.set_entry_point("fetch_data")

# Add edges
# From fetch_data, always go to reasoning
workflow.add_edge("fetch_data", "agent_reasoning")

# Conditional edge from reasoning to decide if it's a tool call or the end
workflow.add_conditional_edges(
    "agent_reasoning",
    decide_next_step,
    {"call_tool": "execute_tools", "end": END}
)

# After executing a tool, cycle back to reasoning to formulate a final summary
workflow.add_edge("execute_tools", "agent_reasoning")

# Compile the graph
app = workflow.compile()

# 6. Run the Agent
print("--- STARTING AD OPTIMIZATION AGENT RUN ---")

# The agent runs autonomously until it hits the END node
# It starts by fetching data, which primes the first message.
final_state = app.invoke(
    {"messages": [], "campaign_data": {}, "next_action": "start"},
    config={"recursion_limit": 50}
)

# 7. Print Final Result
final_message = final_state["messages"][-1].content
print("\n" + "="*50)
print("AGENT FINAL RECOMMENDATION & SUMMARY:")
print(final_message)
print("="*50)

# Optional: Visualize the graph (requires pydot/graphviz)
# from IPython.display import Image
# Image(app.get_graph().draw_png())

--- STARTING AD OPTIMIZATION AGENT RUN ---
--- FETCHING DATA ---
--- AGENT REASONING ---
--- DECIDING NEXT STEP ---
state['next_action']: call_tool
Returning 'call_tool'
--- EXECUTING TOOL ---
--- AGENT REASONING ---
--- DECIDING NEXT STEP ---
state['next_action']: FINISH
Returning 'end'

AGENT FINAL RECOMMENDATION & SUMMARY:
I have paused the following underperforming ad groups: 2, 4, 6, 11, 15, 19, 41, and 49. Additionally, I have requested new creative assets for these campaigns to enhance performance. Awaiting design feedback on the new creatives.
